In [1]:
!pip install pandas scikit-learn imbalanced-learn flask joblib flask-ngrok

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import joblib

# Load the dataset
df = pd.read_csv('creditcard.csv')

# --- NEW STEP ---
# Drop any rows with missing values to ensure clean data for evaluation
df.dropna(inplace=True)

# Separate features and target
X = df.drop('Class', axis=1)
y = df['Class']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Original training data size: {len(X_train)} samples")
print(f"Original test data size: {len(X_test)} samples")
print(f"Original training class distribution:\n{y_train.value_counts()}")

Original training data size: 164845 samples
Original test data size: 41212 samples
Original training class distribution:
Class
0.0    164519
1.0       326
Name: count, dtype: int64


In [7]:
# Scale the 'Time' and 'Amount' features
scaler = StandardScaler()
X_train[['Time', 'Amount']] = scaler.fit_transform(X_train[['Time', 'Amount']])
X_test[['Time', 'Amount']] = scaler.transform(X_test[['Time', 'Amount']])

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print(f"Resampled training data size: {len(X_resampled)} samples")
print(f"Resampled training class distribution:\n{y_resampled.value_counts()}")

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_resampled, y_resampled)

# Save the trained model and the scaler for deployment
joblib.dump(model, 'fraud_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

Resampled training data size: 329038 samples
Resampled training class distribution:
Class
0.0    164519
1.0    164519
Name: count, dtype: int64


['scaler.pkl']

In [8]:
# Make predictions
y_pred = model.predict(X_test)

# Print evaluation metrics
print("--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

--- Confusion Matrix ---
[[41142     3]
 [   10    57]]

--- Classification Report ---
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     41145
         1.0       0.95      0.85      0.90        67

    accuracy                           1.00     41212
   macro avg       0.97      0.93      0.95     41212
weighted avg       1.00      1.00      1.00     41212



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='f1',
                           verbose=2,
                           n_jobs=-1)

# Fit the grid search to the resampled training data
grid_search.fit(X_resampled, y_resampled)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best F1-score: {grid_search.best_score_}")

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test)

# Print the final classification report
print(classification_report(y_test, y_pred_tuned))

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [ ]:
# Save the best model and the scaler
joblib.dump(grid_search.best_estimator_, 'fraud_model_tuned.pkl')
joblib.dump(scaler, 'scaler.pkl')